In [1]:
import argparse
import os
import warnings
import yaml

import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import TQDMProgressBar
from pytorch_lightning.accelerators import find_usable_cuda_devices
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
from nanodet.data.collate import naive_collate
from nanodet.data.dataset import build_dataset
from nanodet.evaluator import build_evaluator
from nanodet.trainer.task import TrainingTask
from torchvision.transforms import ToTensor, ToPILImage
from nanodet.util import (
    NanoDetLightningLogger,
    cfg,
    convert_old_model,
    env_utils,
    load_config,
    load_model_weight,
    mkdir,
)

#Set logger and seed
logger = NanoDetLightningLogger('test')
pl.seed_everything(1234)

Global seed set to 1234


1234

In [2]:
#Function to create the task configuration file required for training
def create_exp_cfg(yml_path, task):
    all_names = ["aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]
    #Load the YAML file
    with open(yml_path, 'r') as file:
        temp_cfg = yaml.safe_load(file)
    #Save dir of the model
    temp_cfg['save_dir'] = 'models/task' + str(task)
    #If base task, training and testing classes are the same
    if task == 0:
        temp_cfg['data']['train']['class_names'] = all_names[:15]
        temp_cfg['data']['val']['class_names'] = all_names[:15]
        temp_cfg['model']['arch']['head']['num_classes'] = 20#15
        temp_cfg['model']['arch']['aux_head']['num_classes'] = 20#15
    #Else, training only on task specific class, and testing on all classes
    else:
        temp_cfg['data']['train']['class_names'] = [all_names[14+task]]
        temp_cfg['data']['val']['class_names'] = all_names[:15+task]
        temp_cfg['model']['arch']['head']['num_classes'] = 20 #15+task
        temp_cfg['model']['arch']['aux_head']['num_classes'] = 20 #15+task
        temp_cfg['schedule']['load_model'] = 'models/task' + str(task-1) + '/model_best/model_best.ckpt'
        
    temp_cfg_name = 'cfg/task' + str(task) + '.yml'
    print(temp_cfg_name)
    #Save the new configuration file
    with open(temp_cfg_name, 'w') as file:
        yaml.safe_dump(temp_cfg, file)

In [ ]:
#Learning stream
#task 0: train on first 15 classes, test on 15 classes
#task 1: train on class n°16, test on 16 classes
#task 2: train on class n°17, test on 17 classes
#task 3: train on class n°18, test on 18 classes
#task 4: train on class n°19, test on 19 classes
#task 5: train on class n°20, test on 20 classes
for task in range (0, 5):
    logger = NanoDetLightningLogger('run_logs/task'+str(task))
    logger.info("Starting task" + str(task))
    logger.info("Setting up data...")
    #Create the task configuration file based on the task number and load the configuration
    create_exp_cfg('cfg/VOC.yml', task)
    load_config(cfg, 'cfg/task' + str(task) + '.yml')
    #Build datasets and dataloaders based on the task configuration file
    train_dataset = build_dataset(cfg.data.train, "train")
    val_dataset = build_dataset(cfg.data.val, "test")
    evaluator = build_evaluator(cfg.evaluator, val_dataset)
    train_dataloader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=cfg.device.batchsize_per_gpu,
        shuffle=True,
        num_workers=cfg.device.workers_per_gpu,
        pin_memory=True,
        collate_fn=naive_collate,
        drop_last=True,
    )
    val_dataloader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=cfg.device.batchsize_per_gpu,
        shuffle=False,
        num_workers=cfg.device.workers_per_gpu,
        pin_memory=True,
        collate_fn=naive_collate,
        drop_last=False,
    )
    #Create the model based on the task configuration file
    logger.info("Creating model...")
    task = TrainingTask(cfg, evaluator)
    #Load the model weights if task is not 0
    if "load_model" in cfg.schedule:
        ckpt = torch.load(cfg.schedule.load_model)
        if "pytorch-lightning_version" not in ckpt:
            warnings.warn(
                "Warning! Old .pth checkpoint is deprecated. "
                "Convert the checkpoint with tools/convert_old_checkpoint.py "
            )
            ckpt = convert_old_model(ckpt)
        load_model_weight(task.model, ckpt, logger)
        logger.info("Loaded model weight from {}".format(cfg.schedule.load_model))
    model_resume_path = (
        os.path.join(cfg.save_dir, "model_last.ckpt")
        if "resume" in cfg.schedule
        else None
    )
    #Set the device to GPU if available
    if cfg.device.gpu_ids == -1:
        logger.info("Using CPU training")
        accelerator, devices, strategy, precision = (
            "cpu",
            None,
            None,
            cfg.device.precision,
        )
    else:
        accelerator, devices, strategy, precision = (
            "gpu",
            cfg.device.gpu_ids,
            None,
            cfg.device.precision,
        )

    if devices and len(devices) > 1:
        strategy = "ddp"
        env_utils.set_multi_processing(distributed=True)

    trainer = pl.Trainer(
        default_root_dir=cfg.save_dir,
        max_epochs=cfg.schedule.total_epochs,
        check_val_every_n_epoch=cfg.schedule.val_intervals,
        accelerator=accelerator,
        devices=[2],
        log_every_n_steps=cfg.log.interval,
        num_sanity_val_steps=0,
        callbacks=[TQDMProgressBar(refresh_rate=0)],
        logger=logger,
        benchmark=cfg.get("cudnn_benchmark", True),
        gradient_clip_val=cfg.get("grad_clip", 0.0),
        strategy=strategy,
        precision=precision,
    )
    trainer.fit(task, train_dataloader, val_dataloader, ckpt_path=model_resume_path)

[NanoDet][02-02 09:55:41]INFO:Starting task0
[NanoDet][02-02 09:55:41]INFO:Starting task0
[NanoDet][02-02 09:55:41]INFO:Setting up data...
[NanoDet][02-02 09:55:41]INFO:Setting up data...


cfg/task0.yml


creating index...
index created!


[NanoDet][02-02 09:55:42]INFO:Creating model...
[NanoDet][02-02 09:55:42]INFO:Creating model...
INFO:NanoDet:Creating model...


creating index...
index created!
model size is  1.0x
init weights...
=> loading pretrained model https://download.pytorch.org/models/shufflenetv2_x1-5666bf0f80.pth
Finish initialize NanoDet-Plus Head.


...

[NanoDet][02-02 11:06:30]INFO:Loaded model weight from models/task0/model_best/model_best.ckpt
[NanoDet][02-02 11:06:30]INFO:Loaded model weight from models/task0/model_best/model_best.ckpt
[NanoDet][02-02 11:06:30]INFO:Loaded model weight from models/task0/model_best/model_best.ckpt
INFO:NanoDet:Loaded model weight from models/task0/model_best/model_best.ckpt
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name      | Type        | Params
------------------------------------------
0 | model     | NanoDetPlus | 4.2 M 
1 | avg_model | NanoDetPlus | 4.2 M 
------------------------------------------
8.4 M     Trainable params
0         Non-trainable params
8.4 M     Total params
33.631    Total estimated model params size (MB)
[NanoDet][02-02 11:06:30]INFO:Weight Averaging is enabled
[NanoDet][02-02 11:06:30]INFO:Weight Averaging is ena

Loading and preparing results...
DONE (t=0.23s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=2.37s).
Accumulating evaluation results...


[NanoDet][02-02 11:12:14]INFO:
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.007
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.016
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.005
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.004
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.011
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.008
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.018
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.022
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.020
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.029

[NanoDet

DONE (t=0.66s).


[NanoDet][02-02 11:12:15]INFO:Saving model to models/task1/model_best/nanodet_model_best.pth
[NanoDet][02-02 11:12:15]INFO:Saving model to models/task1/model_best/nanodet_model_best.pth
[NanoDet][02-02 11:12:15]INFO:Saving model to models/task1/model_best/nanodet_model_best.pth
INFO:NanoDet:Saving model to models/task1/model_best/nanodet_model_best.pth
[NanoDet][02-02 11:12:15]INFO:Val_metrics: {'mAP': 0.007284823444801179, 'AP_50': 0.01603931412210928, 'AP_75': 0.005338377662801417, 'AP_small': 0.00026430143014301424, 'AP_m': 0.0038597868243995916, 'AP_l': 0.01139196293948265}
[NanoDet][02-02 11:12:15]INFO:Val_metrics: {'mAP': 0.007284823444801179, 'AP_50': 0.01603931412210928, 'AP_75': 0.005338377662801417, 'AP_small': 0.00026430143014301424, 'AP_m': 0.0038597868243995916, 'AP_l': 0.01139196293948265}
[NanoDet][02-02 11:12:15]INFO:Val_metrics: {'mAP': 0.007284823444801179, 'AP_50': 0.01603931412210928, 'AP_75': 0.005338377662801417, 'AP_small': 0.00026430143014301424, 'AP_m': 0.00385

In [ ]:
#PROBLEMS
#0. Model AP when training on new task goes immediately to 0
#   Looks like to model is learning pottedplant with the name "aeroplane"
#   xml dataset returns coco_dict, in cocodict need to report all the categories, FIXED
#1. Id of new class need to be consistent with id of validation dataset (sub-rroblem of 0.), FIXED
#2. Evaluator results is not printing on txt file per class AP, FIXED
#3. Weights are loaded from previous model skipping completely the heads.
#   As the model has not always the same number of classes, the model loads its weight but doesn't load the heads, this means that the model
#   immediately forgets the previous task, FIXED
# 5. TO check that the adaptation from one task to another is working, we can check that the new "adpted model" works on the previous tasks as before,This before starting to learn on the new task. -> WORKS BUT NOT PERFECTLY!!